Of course, a language feature would not be worthy of the name "class" without supporting inheritance. The syntax for a derived class definition looks like this:

```py

class DerivedClassName(BaseClassName):
    <statement-1>
    .
    .
    .
    <statement-N>
    pass
```

The name BaseClassName must be defined in a namespace accesible form the scope containing the derived class definition. In place of a base class name, other arbitrary expressions are also allowed. The can be useful, for example, when the base class is defined in another module:

```py

class DerivedClassName(modname.BaseClassName):
    pass
```
Execution of a derived class definition proceeds tha same as for a base class. When the class object is the constructed, the base class is remembered. This is used for resolving attribute references: if a requested attribute is not found in the class, the search proceeds to look in the base class. This rule is applied recursively if the base class itself is derived form some other class.

There's nothing special about instantiation of derived classes: DerivedClassName() creates a new instance of the class. Emthod references are resolved as follows: the corresponding class attibute is searched, descending own the chain of base classes if necessary, and the method reference is valid if the yields a function object.

In [ ]:
# 예제 1: 파생 클래스 인스턴스화 (문장 1, 2)
class Animal:
    def __init__(self, name):
        self.name = name
    
    def speak(self):
        return f"{self.name} makes a sound"

class Dog(Animal):  # Animal을 상속받는 파생 클래스
    def speak(self):
        return f"{self.name} barks"

# 파생 클래스 인스턴스화 - 특별한 것이 없음
my_dog = Dog("Buddy")  # DerivedClassName() 형태
print(my_dog.speak())  # "Buddy barks"


In [2]:
# 예제 2: 메서드 해결 과정 (문장 3, 4, 5)
class Vehicle:
    def start(self):
        return "Vehicle started"
    
    def stop(self):
        return "Vehicle stopped"

class Car(Vehicle):
    def start(self):
        return "Car engine started"

class SportsCar(Car):
    def accelerate(self):
        return "Sports car accelerating"


sports_car = SportsCar()

print(sports_car.accelerate())

print(sports_car.start())

print(sports_car.stop())

print(type(sports_car.start))

Sports car accelerating
Car engine started
Vehicle stopped
<class 'method'>


In [ ]:
# 예제 3: 메서드 해결 순서 시각화
class A:
    def method(self):
        return "A의 메서드"

class B(A):
    def method(self):
        return super().method()

class C(B):
    pass  # method를 오버라이드하지 않음

# 메서드 해결 과정
obj = C()
print("메서드 해결 순서:")
print("1. C 클래스에서 method 검색 -> 없음")
print("2. B 클래스에서 method 검색 -> 발견!")
print(f"결과: {obj.method()}")  # "A의 메서드"
print(obj.method())

# MRO (Method Resolution Order) 확인
print(f"MRO: {C.__mro__}")
# (<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


메서드 해결 순서:
1. C 클래스에서 method 검색 -> 없음
2. B 클래스에서 method 검색 -> 발견!
결과: A의 메서드
A의 메서드
MRO: (<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


Derived classes may override methods of their base classes. Because methods have no special privileges when calling other methods of the same object, a method of a base class that calls another method defined in the same base that cllas another method defined in the same base class may end up calling a method of a derived class that overrides it. (For C++ programmers: all methods in Python are effectively virtual)

In [8]:
# 예제 4: 동적 메서드 바인딩 기본 예제
# 부모 클래스의 메서드가 다른 메서드를 호출할 때, 자식의 오버라이드된 메서드가 실행됨

class BaseClass:
    def method1(self):
        print("BaseClass.method1() 호출됨")
        return self.method2()  # 같은 클래스의 다른 메서드 호출
    
    def method2(self):
        print("BaseClass.method2() 호출됨")
        return "기본 클래스의 결과"

class DerivedClass(BaseClass):
    def method2(self):  # method1은 오버라이드하지 않고 method2만 오버라이드
        print("DerivedClass.method2() 호출됨")
        return "파생 클래스의 결과"

# 테스트
print("=== 기본 클래스 인스턴스 ===")
base_obj = BaseClass()
result = base_obj.method1()
print(f"최종 결과: {result}")

print("\n=== 파생 클래스 인스턴스 ===")
derived_obj = DerivedClass()
result = derived_obj.method1()  # method1을 호출했지만
print(f"최종 결과: {result}")

print("\n핵심: 부모의 method1()이 method2()를 호출할 때, 실제 객체가 DerivedClass이므로")
print("DerivedClass.method2()가 실행됩니다!")


=== 기본 클래스 인스턴스 ===
BaseClass.method1() 호출됨
BaseClass.method2() 호출됨
최종 결과: 기본 클래스의 결과

=== 파생 클래스 인스턴스 ===
BaseClass.method1() 호출됨
DerivedClass.method2() 호출됨
최종 결과: 파생 클래스의 결과

핵심: 부모의 method1()이 method2()를 호출할 때, 실제 객체가 DerivedClass이므로
DerivedClass.method2()가 실행됩니다!


In [9]:
# 예제 5: 템플릿 메서드 패턴을 통한 실용적 예제
# 부모 클래스가 여러 단계의 메서드를 호출하는데, 각 단계를 자식에서 커스터마이즈 가능

class DocumentProcessor:
    def process_document(self, content):
        """문서 처리의 전체 흐름 (템플릿 메서드)"""
        print("=== 문서 처리 시작 ===")
        
        # 각 단계를 순서대로 실행
        validated = self.validate_content(content)
        formatted = self.format_content(validated)
        result = self.finalize_document(formatted)
        
        print("=== 문서 처리 완료 ===")
        return result
    
    def validate_content(self, content):
        print("기본 검증 수행")
        return content.strip()

    def format_content(self, content):
        print("기본 포맷팅 수행")
        return content.upper()
    
    def finalize_document(self, content):
        print("기본 최종화 수행")
        return f"[Document] {content}"


class HTMLProcessor(DocumentProcessor):
    def format_content(self, content):
        print("HTML 포맷팅 수행")
        return f"<html><body>{content}</body></html>"

    def finalize_document(self, content):
        print("HTML 최종화 수행")
        return f"<!DOCTYPE html>{content}"


class MarkdownProcessor(DocumentProcessor):
    def validate_content(self, content):
        print("마크다운 검증 수행")
        return content.replace('\n', ' ')
    
    def format_content(self, content):
        print("마크다운 포맷팅 수행")
        return f"# {content}"

# 테스트
print("=== 기본 문서 처리기 ===")
basic_processor = DocumentProcessor()
result1 = basic_processor.process_document("  hello world  ")
print(f"결과: {result1}")

print("\n=== HTML 문서 처리기 ===")
html_processor = HTMLProcessor()
result2 = html_processor.process_document("  hello world  ")
print(f"결과: {result2}")

print("\n=== 마크다운 문서 처리기 ===")
markdown_processor = MarkdownProcessor()
result3 = markdown_processor.process_document("hello\nworld")
print(f"결과: {result3}")

print("\n핵심: 부모의 process_document()가 각 단계 메서드를 호출할 때,")
print("실제 객체 타입에 따라 오버라이드된 메서드들이 실행됩니다!")


=== 기본 문서 처리기 ===
=== 문서 처리 시작 ===
기본 검증 수행
기본 포맷팅 수행
기본 최종화 수행
=== 문서 처리 완료 ===
결과: [Document] HELLO WORLD

=== HTML 문서 처리기 ===
=== 문서 처리 시작 ===
기본 검증 수행
HTML 포맷팅 수행
HTML 최종화 수행
=== 문서 처리 완료 ===
결과: <!DOCTYPE html><html><body>hello world</body></html>

=== 마크다운 문서 처리기 ===
=== 문서 처리 시작 ===
마크다운 검증 수행
마크다운 포맷팅 수행
기본 최종화 수행
=== 문서 처리 완료 ===
결과: [Document] # hello world

핵심: 부모의 process_document()가 각 단계 메서드를 호출할 때,
실제 객체 타입에 따라 오버라이드된 메서드들이 실행됩니다!


In [ ]:
# 예제 6: C++ 비교 및 super() 사용법
# 파이썬의 모든 메서드는 기본적으로 virtual (동적 바인딩)
# virtual과 virtual아 이닌 것의 차이가 뭐길래? 

class BaseCalculator:
    def calculate(self, x, y):
        print("BaseCalculator.calculate() 호출")
        return self.add(x, y)  # 자식의 오버라이드된 add()가 호출됨
    
    def add(self, x, y):
        print("BaseCalculator.add() 호출")
        return x + y

class AdvancedCalculator(BaseCalculator):
    def add(self, x, y):
        print("AdvancedCalculator.add() 호출")
        # 부모의 원래 add()를 명시적으로 호출하고 싶다면 super() 사용
        base_result = super().add(x, y)
        return base_result * 2  # 부모 결과에 2를 곱함

class ScientificCalculator(BaseCalculator):
    def add(self, x, y):
        print("ScientificCalculator.add() 호출")
        # 부모 메서드를 전혀 호출하지 않고 완전히 새로운 구현
        return x + y + 0.1  # 과학적 계산에서는 소수점 오차 고려

class BaseCalculator:
    def calculate(self, x, y):
        print("BaseCalculator.calculate() 호출")
        return self.add(x, y)
    
    def add(self, x, y):
        print("BaseCalculator.add() 호출")
        return sum([x, y])

# 테스트
print("=== 기본 계산기 ===")
basic = BaseCalculator()
result1 = basic.calculate(5, 3)
print(f"결과: {result1}")

print("\n=== 고급 계산기 (super() 사용) ===")
advanced = AdvancedCalculator()
result2 = advanced.calculate(5, 3)
print(f"결과: {result2}")

print("\n=== 과학 계산기 (완전 오버라이드) ===")
scientific = ScientificCalculator()
result3 = scientific.calculate(5, 3)
print(f"결과: {result3}")

print("\n=== C++와의 차이점 ===")
print("C++에서는 virtual 키워드가 필요하지만, 파이썬은 모든 메서드가 기본적으로 virtual입니다.")
print("부모 메서드를 명시적으로 호출하려면 super()를 사용합니다.")
print("super() 없이 self.add()를 호출하면 무한 재귀가 발생할 수 있습니다!")

# 무한 재귀 예시 (실행하지 않음)
print("\n# 주의: 이런 코드는 무한 재귀를 일으킵니다!")
print("# def add(self, x, y):")
print("#     return self.add(x, y)  # 자기 자신을 호출!")


=== 기본 계산기 ===
BaseCalculator.calculate() 호출
BaseCalculator.add() 호출
결과: 8


An overriding method in a derived class may in fact want to extend rather than simply replace the base class method of the same name. There is a simple way to call the base class method directly. just call BaseClassName.methodname(self, arguments). This is occasionally useful to clients as well. (Note that this only works if the base class is accessible as BaseClassName in the global scope.)

Python has two built-in functions that work with inheritance:

- Use isinstance() to check an instance's type: istinstance(obj, int) will be True if obj.__class__ is int or some class derived from int.

- Use issubclass() to check class inheritance: issubclass(bool, int) is True since bool is a subclass of int. However, issubclass(float, int) is False since float is not subclass of int



9.5.1. Multiple inheritance

Python supports a form of multiple inheritance as well. A calss definetion with multiple base classes look like this:

```py

class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <tatemkent-N>

```

For most purposes, in the simplest cases, you can think of the search for attributes inheritance from a parent class as depth-first, left-to-right, not searching twice in the same class where there is overlap in the hiercarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was serached for in Base2, and so on.

In fact, it is slightly more complex than that; the method resolution order changes dynamically to support cooperative calls to super(). This approach is known in some other This approach is known in some other multiple-inheritance languages as call-next-method and is more powerful than the super call found in single-inheritance languages.


In [12]:
# "cooperative calls to super()": 
# in multiple-inheritance, super() not merely call their parent class. can call next class of Method Resolution Order
# super() is not parent actually next participant of next MRO chain
class A:
    def method(self):
        print("A")
    
class B(A):
    def method(self):
        print("B")
        super().method()
    
class C(A):
    def method(self):
        print("C")
        super().method()

class D(B, C):
    def method(self):
        print("D")
        super().method()

D().method()

D
B
C
A


Dynamic ordering is necessary because all cases of multiple inheritance exhibit one or more diamond relationship(where at leat one of the parent classes can be accessed through multiple paths from the bottommost class). For example, all classes inherit from object, so any case of multiple inheritance provdes more than one path to reach object. To keep the base classes from being accessd more than once, the dynamic algorithm linearizes the search order in a way that preserves the left-to-right ordering specified in each class, that calls each parent only once, and that is monotonic (meaning that a clas can be subcalssed without affecting the precedence order of its parents). Taken together, these properties make it possible to design reliable and extensible calsses with multiple inheritance, For more detail, see The Python 2.3 Method Resolution Order. 

9.6. Private Variables

"Private" instance variables that cannot be accessed except from inside an object dont' exist in Python. Howver, there is a convention that is followed byh most Python code: a name prefixed with an underscore (e.g, _spam) should be treated as a non-public part of the API (whether it is a function, a method or a data member). It should be considered an implementation detail and subject to change without notice.

Since there is a valid use-case for calss-private members (namely to avoid name clashes of names with names defined by subclasses). 